In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import math

In [96]:
with open('input.txt') as inp:
    in_map = inp.read()

In [4]:
test_map = '''.#..#
.....
#####
....#
...##'''

In [9]:
def matrix_from_map(map_str):
    rows = map_str.split('\n')
    arrays = []
    for row in rows:
        arrays.append(np.array([[np.nan if c is '.' else 1 for c in row ]]))
    return np.concatenate(arrays)
        

In [6]:
ast_matrix = matrix_from_map(in_map)
wid = np.size(ast_matrix, 0)
hei = np.size(ast_matrix, 1)

## Part I

Since it does not take any time at all, even for the puzzle input, we choose a lazy looping scheme

In [6]:
def generate_vis_matrix(ast_matrix):
    vis_matrix = ast_matrix.copy()
    for i in range(hei):
        for j in range(wid):
            angle_matrix = ast_matrix.copy()
            if ~np.isnan(ast_matrix[i, j]):
                for ia in range(hei):
                    for ja in range(wid):
                        if (i == ia) and (j == ja):
                            angle_matrix[ia, ja] = np.nan
                        else:
                            angle_matrix[ia, ja] *= math.atan2((ja - j), (ia - i))
                vis_matrix[i, j] = len(np.unique(angle_matrix[~np.isnan(angle_matrix)]))
    return vis_matrix

In [171]:
vis_matrix = generate_vis_matrix(ast_matrix)

In [9]:
# maximum number of visible asteroids
np.max(vis_matrix[~np.isnan(vis_matrix)])

214.0

In [10]:
# linear position of max
np.nanargmax(vis_matrix)

328

In [104]:
# i, j position of max
np.unravel_index(np.nanargmax(vis_matrix), (hei, wid))

(16, 8)

## Part II

In [64]:
test_map = '''.#..#
.....
#####
....#
...##'''

In [127]:
test_map = '''.#....#####...#..
##...##.#####..##
##...#...#.#####.
..#.........###..
..#.#.....#....##'''

In [10]:
test_map = '''.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##'''

In [97]:
ast_matrix = matrix_from_map(in_map)
wid = np.size(ast_matrix, 1)
hei = np.size(ast_matrix, 0)

In [98]:
vis_matrix = generate_vis_matrix(ast_matrix)

In [99]:
i, j  = np.unravel_index(np.nanargmax(vis_matrix), (wid, hei))

In [100]:
i, j

(16, 8)

In [101]:
def make_angle_and_dist(ast_matrix, i, j):
    angle_matrix = ast_matrix.copy()
    dist_matrix = ast_matrix.copy()
    for ia in range(hei):
        for ja in range(wid):
            if (i == ia) and (j == ja):
                angle_matrix[ia, ja] = np.nan
                dist_matrix[ia, ja] = np.nan
            else:
                angle_matrix[ia, ja] *= math.atan2((ia - i), (ja - j)) * 180/math.pi
                dist_matrix[ia, ja] = math.sqrt((ja-j)**2 + (ia-i)**2 )
    return angle_matrix, dist_matrix

In [102]:
angle_matrix, dist_matrix = make_angle_and_dist(ast_matrix, i, j)

order_matrix = angle_matrix.copy()
order_matrix[~np.isnan(order_matrix)] = 0

laser_angle = -90

asteroid_number = 1
while 0 in order_matrix:
    diff_m = angle_matrix - laser_angle
    diff_m[diff_m < 0] = np.inf
    rw, cl = np.where(diff_m == np.nanmin(diff_m))
    pts = [(i,j) for i,j in zip(rw, cl)]
    
    
    d_closest_ast = np.inf
    for pt in pts:
        if d_closest_ast > dist_matrix[pt]:
            d_closest_ast = dist_matrix[pt]
            closest_ast = pt
    
    
    laser_angle = angle_matrix[pt] if len(pts) == 1 else angle_matrix[pt]+1e-12
    if laser_angle > 180:
        laser_angle = -179.999999999999999999
    
    angle_matrix[closest_ast] = np.nan
    order_matrix[closest_ast] = asteroid_number
    asteroid_number += 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in less
  # This is added back by InteractiveShellApp.init_path()


In [103]:
np.where(order_matrix == 200)

(array([2], dtype=int64), array([5], dtype=int64))

In [89]:
angle_matrix, dist_matrix = make_angle_and_dist(ast_matrix, i, j)

In [84]:
def map_from_matrix(matrix):
    for i in range(np.size(matrix, 0)):
        row = matrix[i]
        row_list = [str((round(elem, 3))) if not np.isnan(elem) else '.' for elem in row]
        row_str = ''
        for i in row_list:
            row_str+=i+' '*(6-len(i))
        print(row_str)

In [68]:
map_from_matrix(order_matrix)

.    261  .    .    269  271  .    277  281  284  .    .    .    7    11   16   20   25   27   33   
253  258  .    263  266  296  273  276  280  283  286  297  2    288  290  .    .    298  31   .    
.    254  .    262  264  268  272  275  279  .    285  287  3    9    13   18   24   28   .    38   
.    299  255  259  .    265  270  274  278  282  236  230  .    289  15   291  292  .    36   .    
248  251  295  256  260  .    267  226  .    219  .    217  4    .    17   23   29   .    40   45   
.    .    250  252  257  231  223  .    .    206  .    202  5    12   19   214  34   232  44   293  
244  246  247  249  228  220  212  203  199  198  193  192  6    14   22   30   37   43   229  47   
243  .    245  233  221  210  .    .    .    .    184  183  8    .    26   .    42   .    48   234  
241  242  .    224  213  201  194  187  181  177  176  173  10   21   32   41   195  49   52   54   
239  240  227  215  204  .    188  179  .    171  169  167  .    .    39   180  50   53   .

In [90]:
map_from_matrix(angle_matrix)

.     -127.569.     .     -118.301-114.775.     -107.103-102.995-98.746.     .     .     -81.254-77.005-72.897-68.962-65.225-61.699-58.392
-132.51-129.806.     -123.69-120.256-116.565-112.62-108.435-104.036-99.462-94.764-90.0 -85.236-80.538-75.964.     .     -63.435-59.744.     
.     -132.274.     -126.027-122.471-118.61-114.444-109.983-105.255.     -95.194-90.0 -84.806-79.695-74.745-70.017-65.556-61.39.     -53.973
.     -135.0-131.987-128.66.     -120.964-116.565-111.801-106.699-101.31-95.711-90.0 .     -78.69-73.301-68.199-63.435.     -55.008.     
-140.711-138.013-135.0-131.634-127.875.     -119.055-113.962.     -102.529.     -90.0 -83.66.     -71.565-66.038-60.945.     -52.125-48.366
.     .     -138.366-135.0-131.186-126.87-122.005.     .     -104.036.     -90.0 -82.875-75.964-69.444-63.435-57.995-53.13-48.814-45.0 
-147.529-145.008-142.125-138.814-135.0-130.601-125.538-119.745-113.199-105.945-98.13-90.0 -81.87-74.055-66.801-60.255-54.462-49.399-45.0 -41.186
-151.39.     -146.31